In [2]:
import csv
#import matplotlib.pyplot as plt
#import matplotlib.ticker as ticker
import numpy as np
import pandas as pd
import time
from sklearn.externals import joblib
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
label =100
from sklearn.model_selection import GridSearchCV, KFold

In [12]:
data_raw = pd.read_csv('../Documents/shuffled-full-set-hashed.csv',header = None)

In [14]:
data_raw.columns = ['label','features']

In [6]:
from sklearn.utils import shuffle

In [7]:
data_raw = shuffle(data_raw).reset_index()

In [7]:
data_raw = data_raw.drop('index',axis=1)

In [8]:
data_raw.head()

,label,features
0,CANCELLATION NOTICE,a004fb5e5716 c06be6a6100b 365a9390d225 e943e5e...
1,REINSTATEMENT NOTICE,20127286030f 071206f2bcbb 1e1cf4f1b953 f250d72...
2,CANCELLATION NOTICE,9b16d62c7990 5cd2ff1e0fb1 fbe267908bc5 4b45c77...
3,BINDER,bf15989af17d d63be9e66da8 5c2db045bc17 f49ab97...
4,POLICY CHANGE,566ed917bca5 8b0131ee1005 46c88d9303da 8d1bd25...


In [16]:
labels, index = data_raw['label'].factorize()
print('There are {} distinct categories.'.format(len(index)))

There are 14 distinct categories.


In [17]:
labels

array([0, 1, 2, ..., 2, 4, 4])

In [18]:
index

Index(['DELETION OF INTEREST', 'RETURNED CHECK', 'BILL', 'POLICY CHANGE',
       'CANCELLATION NOTICE', 'DECLARATION', 'CHANGE ENDORSEMENT',
       'NON-RENEWAL NOTICE', 'BINDER', 'REINSTATEMENT NOTICE',
       'EXPIRATION NOTICE', 'INTENT TO CANCEL NOTICE', 'APPLICATION',
       'BILL BINDER'],
      dtype='object')

In [19]:
data_raw['label'] = labels
data_raw.head(10)

,label,features
0,0,e04a09c87692 d6b72e591b91 5d066f0246f1 ed41171...
1,1,a3b334c6eefd be95012ebf2b 41d67080e078 ff1c26e...
2,2,586242498a88 9ccf259ca087 54709b24b45f 6bf9c0c...
3,2,cd50e861f48b 6ca2dd348663 d38820625542 f077614...
4,2,9db5536263d8 1c303d15eb65 3f89b4673455 b73e657...
5,3,10e45001c2f2 6a01047db3ab 6a8e3499dab9 97b6014...
6,3,4e5019f629a9 54fb196d55ce 0cf4049f1c7c ef4ea27...
7,2,ad4440ac97a5 8e93a2273a93 c913f5129fe2 bfb030c...
8,4,25c57acdf805 b7a0f56f6ce8 cf4fc632eed2 67f9c02...
9,2,48d657cd9861 fee60001e67a 4ce4bfb42e22 de9738e...


In [20]:
data_raw['features'] = data_raw['features'].astype('str')

In [22]:
data_raw['features'].head()

0    e04a09c87692 d6b72e591b91 5d066f0246f1 ed41171...
1    a3b334c6eefd be95012ebf2b 41d67080e078 ff1c26e...
2    586242498a88 9ccf259ca087 54709b24b45f 6bf9c0c...
3    cd50e861f48b 6ca2dd348663 d38820625542 f077614...
4    9db5536263d8 1c303d15eb65 3f89b4673455 b73e657...
Name: features, dtype: object

In [12]:
X_all = data_raw['features']
Y_all = data_raw['label']
assert len(X_all)==len(Y_all)

In [13]:
cv = CountVectorizer()
tfidf = TfidfTransformer()
clf = MultinomialNB(alpha=0.01)

In [14]:
from sklearn.pipeline import Pipeline

In [15]:
pipeline = Pipeline([('cv',cv),('tfidf',tfidf),('clf',clf)])

In [16]:
#this step is actually meaningless, because the step behind(grid serach) contains this step
kf = KFold(n_splits=5)
accuracy = []
for train_index, val_index in kf.split(X_all):
    X_train, X_val = X_all[train_index], X_all[val_index]
    Y_train, Y_val = Y_all[train_index], Y_all[val_index]
    pipeline.fit(X_train,Y_train)
    prediction = pipeline.predict(X_val)
    ground_truth = Y_val.get_values()
    accuracy.append(np.mean(prediction==ground_truth))
print(accuracy)

[0.7689092516678724, 0.7759826380516036, 0.7771079495217427, 0.7709187364359779, 0.7795819935691318]


In [17]:
parameters = {'cv__max_features':(10000,50000,100000,None),
             'clf__alpha':(1e-2,1e-3)}
gs_clf = GridSearchCV(pipeline,parameters)
gs_clf.fit(X_all,Y_all)
print(gs_clf.best_score_)
print(gs_clf.best_params_)
# accuracy on validation set 78.2% 

0.782682785672947
{'clf__alpha': 0.01, 'cv__max_features': 50000}


In [18]:
# use train model to test all training data 84%
total_prediction = gs_clf.predict(X_all)

In [19]:
np.mean(total_prediction==Y_all.get_values())

0.8487235547553212

In [20]:
#用目前最优的参数重新初始化所有transformer和estimator,直接在所有data上训练，得到的结果和上面一样
cv = CountVectorizer(max_features=50000)
tfidf = TfidfTransformer()
clf = MultinomialNB(alpha=0.01)
pipeline = Pipeline([('cv',cv),('tfidf',tfidf),('clf',clf)])
pipeline.fit(X_all,Y_all)
pred = pipeline.predict(X_all)
np.mean(pred==Y_all.get_values())

0.8487235547553212

In [24]:
with open('nb_model.z', 'wb') as f:
                joblib.dump(pipeline, f, protocol=2)

In [28]:
feature_all = tfidf.fit_transform(cv.fit_transform(X_all))

In [29]:
feature_all

<62204x1000 sparse matrix of type '<type 'numpy.float64'>'
	with 6916879 stored elements in Compressed Sparse Row format>

In [37]:
X_test, Y_test = feature_all[:1000], Y_all[:1000]
X_rest, Y_rest = feature_all[1000:], Y_all[1000:]
Y_rest = Y_rest.reset_index(drop=True)

In [66]:
kf = KFold(n_splits=10)
clf = MultinomialNB(alpha=0.01)
accuracy = []
for train_index, val_index in kf.split(X_rest):
    X_train, X_val = X_rest[train_index], X_rest[val_index]
    Y_train, Y_val = Y_rest[train_index], Y_rest[val_index]
    clf.fit(X_train,Y_train)
    prediction = clf.predict(X_val)
    ground_truth = Y_val.get_values()
    accuracy.append(np.mean(prediction==ground_truth))
print(accuracy)

[0.7761803626858357, 0.7716059467407287, 0.7753635026956379, 0.7714425747426891, 0.7772875816993464, 0.7676470588235295, 0.7733660130718955, 0.7697712418300654, 0.7766339869281046, 0.7728758169934641]


In [67]:
prediction = clf.predict(X_test)
test_acc = np.mean(prediction==Y_test.get_values())

In [68]:
test_acc

0.788